In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix
from collections import defaultdict


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Annotated resutls/CSV/iaa_calculationconfusion_sy.csv')
data

,instance_id,evaluator_topics,evaluator_sentiments,annotator1topics,annotator1sentiments,annotator2topics,annotator2sentiments
0,1,3.1.3,5,2.2.1,5,3.1.3,5
1,2,"2.2.2, 4.4.4","4, 1","4.3.2, 4.5.2, 4.3.3","4, 4, 5","3.2.1, 3.2.3","5, 5"
2,3,7.1.1,5,7.1.1,5,7.1.1,5
3,4,"2.2.1, 2.2.4, 4.1.3","5, 1, 1",2.2.4,1,4.1.3,5
4,5,1.2.1,2,1.2.1,2,1.2.2,1
...,...,...,...,...,...,...,...
1795,1796,"2.1.1, 1.3.1, 2.1.2","2, 2, 1","2.1.1, 2.1.2","1, 1","2.1.1, 2.1.2","1, 1"
1796,1797,7.2.4,1,"3.1.4, 7.2.4, 7.3.3","1, 1, 1",7.2.1,2
1797,1798,5.2.5,3,6.1.1,3,6.1.1,2
1798,1799,"7.2.2, 1.3.5","5, 4","1.3.5, 4.2.2","3, 5",2.2.9,5


In [ ]:
df = pd.DataFrame({
                       'instance_id': data["instance_id"],
                       'annotator_1': data["annotator1topics"],
                       'annotator_2': data["annotator2topics"]})
df

,instance_id,annotator_1,annotator_2
0,1,2.2.1,3.1.3
1,2,"4.3.2, 4.5.2, 4.3.3","3.2.1, 3.2.3"
2,3,7.1.1,7.1.1
3,4,2.2.4,4.1.3
4,5,1.2.1,1.2.2
...,...,...,...
1795,1796,"2.1.1, 2.1.2","2.1.1, 2.1.2"
1796,1797,"3.1.4, 7.2.4, 7.3.3",7.2.1
1797,1798,6.1.1,6.1.1
1798,1799,"1.3.5, 4.2.2",2.2.9


In [ ]:
df['annotator_1'] = df['annotator_1'].apply(lambda x: x.split(', '))
df['annotator_2'] = df['annotator_2'].apply(lambda x: x.split(', '))

unique_classifications = set()
for labels in pd.concat([df['annotator_1'], df['annotator_2']]):
    unique_classifications.update(labels)

results = []

for target_classification in unique_classifications:
    mix_counts = defaultdict(int)

    for _, row in df.iterrows():
        annotator_1_labels = set(row['annotator_1'])
        annotator_2_labels = set(row['annotator_2'])

        if target_classification in annotator_1_labels:
            for label in annotator_1_labels:
                if label != target_classification:
                    mix_counts[label] += 1

        if target_classification in annotator_2_labels:
            for label in annotator_2_labels:
                if label != target_classification:
                    mix_counts[label] += 1

    mix_df = pd.DataFrame(mix_counts.items(), columns=['Mixed Classification', 'Count'])

    if not mix_df.empty:
        top_mixed = mix_df.nlargest(1, 'Count')
        top_mixed['Target Classification'] = target_classification
        results.append(top_mixed[['Target Classification', 'Mixed Classification', 'Count']])

final_df = pd.concat(results).reset_index(drop=True)

print(final_df)

    Target Classification Mixed Classification  Count
0                   5.1.1                2.1.2      5
1                   7.3.1                7.2.1      3
2                   1.1.3                4.6.7      8
3                   6.3.3                6.3.1      3
4                   3.1.1                3.2.1     40
..                    ...                  ...    ...
108                2.2.14                2.2.8      3
109                 4.3.3                4.3.5     12
110                 4.2.2                4.2.1     15
111                 7.2.4                7.2.1      7
112                 4.3.4                4.3.3      4

[113 rows x 3 columns]


In [ ]:
final_df.to_csv('out_1.csv')